In [1]:
import json, pycm, pandas as pd
from nl_classifier import NaturalLanguageClassifer
from datetime import datetime

In [2]:
MODEL_NAME = 'gpt-4'
TEMPERATURE = 0.1
RESULTS_FILENAME = 'planet_gpt-4_description_results.json'

In [3]:
experiments = json.load(open(RESULTS_FILENAME, 'r'))

In [4]:
for i, experiment in enumerate(experiments):
    cls = experiment["concept"]
    classifier = NaturalLanguageClassifer(cls["id"], cls["label"], cls["definition"], MODEL_NAME, TEMPERATURE)
    print(f'{i+1:02}: {cls["label"]:30} {cls["id"]}')
    data = experiment["data"]
    for j, entity in enumerate(data):
        if "predicted" in entity:
            print(f'   {j+1:02}: {entity["label"]:30} {entity["id"]:45} SKIPPING')
        else:
            print(f'   {j+1:02}: {entity["label"]:30} {entity["id"]:45} {entity["actual"]:10} {classifier.tokens_used(entity["label"] + " " + entity["description"]):05} tokens ', end=" ")
            classification = classifier.classify(entity["label"], entity["description"])
            entity["predicted"] = classification["predicted"].lower()
            if entity["actual"] != entity["predicted"]:
                if entity["actual"] == "positive":
                    print("FN")
                else:
                    print("FP")
            else:
                print("")
            entity["rationale"] = classification["rationale"]
        df_results = pd.DataFrame.from_records(data)
        cm = pycm.ConfusionMatrix(df_results["actual"].tolist(), df_results["predicted"].tolist(), digit=2, classes=[ 'positive', 'negative' ])
        experiment["confusion_matrix"] = cm.matrix
        experiment["created"] = datetime.now().isoformat()
        json.dump(experiments, open(RESULTS_FILENAME, 'w+'))

01: planet                         iau_2006_08_24_planet
   01: HD 159868 b                    http://www.wikidata.org/entity/Q841870        SKIPPING
   02: HD 102272 c                    http://www.wikidata.org/entity/Q849182        SKIPPING
   03: HD 118203 b                    http://www.wikidata.org/entity/Q849432        SKIPPING
   04: DP Leonis b                    http://www.wikidata.org/entity/Q858156        SKIPPING
   05: DENIS-P J082303.1−491201 b     http://www.wikidata.org/entity/Q17010263      SKIPPING
   06: WD 0806-661 b                  http://www.wikidata.org/entity/Q9369324       SKIPPING
   07: Kepler-56c                     http://www.wikidata.org/entity/Q19672944      SKIPPING
   08: Kepler-560b                    http://www.wikidata.org/entity/Q28855701      SKIPPING
   09: HD 23596 b                     http://www.wikidata.org/entity/Q849606        SKIPPING
   10: HD 202206 b                    http://www.wikidata.org/entity/Q9284610       SKIPPING
   11: XO-4b 